**1. Read data**


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
#import torch
import json

# Đọc dữ liệu từ tệp tin JSON
with open('zac2022_train_merged_final.json', 'rb') as f:
    data = json.load(f)

data = data["data"]

# New Section

In [3]:
full_data = []
dict = {}
for item in data:
  if item['category'] == "FULL_ANNOTATION":
    full_data.append(item)


print(len(full_data))
#print(full_data)

4989


In [4]:
for obj in full_data:
    del obj["short_candidate_start"]
    del obj["answer"]
    del obj["category"]
    del obj["is_long_answer"]
    del obj["title"]
    #del obj["id"]
print(full_data[2])

{'id': 'ee8a272bcefac7b96037a1571984b1f8', 'question': 'Thành phố nào là thủ phủ của Ai Cập trong đế quốc La Mã', 'text': 'Lịch sử . Ai Cập bị La Mã chiếm năm 30 TCN , và Alexandria trở thành thủ phủ của tỉnh Ai Cập trong đế quốc La Mã .', 'short_candidate': 'Alexandria'}


a. Tìm vị trí đầu của answer(short_candidate) trong text

In [5]:
def get_start_idx(story):
    str_idx = story['text'].find(story['short_candidate'])
    return {'str_idx':str_idx}

In [6]:
for item in full_data:
  item['str_idx'] = get_start_idx(item)['str_idx']

print(full_data[0]['str_idx'])

53


Đưa dạng data về datasets đồng thời chia làm hai phần train và test

In [ ]:
!pip install datasets
from datasets import DatasetDict

In [8]:
data_train = full_data
train_ds = data_train[:4000]
test_ds = data_train[4000:]

In [9]:
qa_dataset = DatasetDict()
qa_dataset.update({"train": train_ds})
qa_dataset.update({"test": test_ds})

In [10]:
import pandas as pd
from datasets import Dataset

# Your list of dictionaries

# Create a pandas DataFrame from your list
my_dataframe1 = pd.DataFrame(train_ds)
my_dataframe2 = pd.DataFrame(test_ds)

# Create a dataset from your dataframe
train_ds = Dataset.from_pandas(my_dataframe1)
test_ds = Dataset.from_pandas(my_dataframe2)
print(type(test_ds))

<class 'datasets.arrow_dataset.Dataset'>


**2. Tokenize**

In [ ]:
!pip install git+https://github.com/datquocnguyen/transformers

In [12]:
from transformers import BartphoTokenizerFast

tokenizer = BartphoTokenizerFast.from_pretrained("vinai/bartpho-word-base")
#tokenizer.is_fast

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'PhobertTokenizer'. 
The class this function is called from is 'BartphoTokenizerFast'.


In [13]:
text = train_ds[0]["text"]
question = train_ds[0]["question"]

inputs = tokenizer(question,text)
tokenizer.decode(inputs["input_ids"])

'<s>Đất nước nào không có quân đội </s></s>có 23 quốc gia không có lực lượng quân đội, bao gồm: Costa Rica, Iceland, Panama, Micronesia, Quần đảo Marshall, và Vatican... </s>'

In [14]:
inputs = tokenizer(
    question,
    text,
    max_length=100,
    truncation="only_second",
    stride=50,
    return_overflowing_tokens=True,
)

for ids in inputs["input_ids"]:
    print(tokenizer.decode(ids))
    print()

<s>Đất nước nào không có quân đội </s></s>có 23 quốc gia không có lực lượng quân đội, bao gồm: Costa Rica, Iceland, Panama, Micronesia, Quần đảo Marshall, và Vatican... </s>



`max_length` to set the maximum length (here 100)

`truncation="only_second"` to truncate the context (which is in the second position) when the question with its context is too long

`stride` to set the number of overlapping tokens between two successive chunks (here 50)

`return_overflowing_tokens=True` to let the tokenizer know we want the overflowing tokens

In [15]:
inputs = tokenizer(
    question,
    text,
    max_length=100,
    truncation="only_second",
    stride=50,
    return_overflowing_tokens=True,
    return_offsets_mapping=True,
)
inputs.keys()

dict_keys(['input_ids', 'attention_mask', 'offset_mapping', 'overflow_to_sample_mapping'])

In [16]:
inputs["overflow_to_sample_mapping"]

[0]

In [17]:
inputs = tokenizer(
    train_ds[2:5]["question"],
    train_ds[2:5]["text"],
    max_length=100,
    truncation="only_second",
    stride=50,
    return_overflowing_tokens=True,
    return_offsets_mapping=True,
)

print(f"The 3 examples gave {len(inputs['input_ids'])} features.")
print(f"Here is where each comes from: {inputs['overflow_to_sample_mapping']}.")

The 3 examples gave 5 features.
Here is where each comes from: [0, 1, 2, 2, 2].


In [ ]:
#train_ds[2:5]

`sequence_ids` similar to Next Sentence Prediction

In [19]:
answers = train_ds[2:5]["short_candidate"]
idxs = train_ds[2:5]["str_idx"]
start_positions = []
end_positions = []

for i, offset in enumerate(inputs["offset_mapping"]):
    sample_idx = inputs["overflow_to_sample_mapping"][i]
    answer = answers[sample_idx]
    start_char = idxs[sample_idx]
    end_char = start_char + len(answer)
    sequence_ids = inputs.sequence_ids(i)

    # Find the start and end of the context
    idx = 0
    while sequence_ids[idx] != 1:
        idx += 1
    context_start = idx
    while sequence_ids[idx] == 1:
        idx += 1
    context_end = idx - 1

    # If the answer is not fully inside the context, label is (0, 0)
    if offset[context_start][0] > start_char or offset[context_end][1] < end_char:
        start_positions.append(0)
        end_positions.append(0)
    else:
        # Otherwise it's the start and end token positions
        idx = context_start
        while idx <= context_end and offset[idx][0] <= start_char:
            idx += 1
        start_positions.append(idx - 1)

        idx = context_end
        while idx >= context_start and offset[idx][1] >= end_char:
            idx -= 1
        end_positions.append(idx + 1)

start_positions, end_positions

([33, 32, 84, 48, 0], [33, 33, 84, 48, 0])

In [20]:
idx = 0
sample_idx = inputs["overflow_to_sample_mapping"][idx]
answer = answers[sample_idx]

start = start_positions[idx]
end = end_positions[idx]
labeled_answer = tokenizer.decode(inputs["input_ids"][idx][start : end + 1])

print(f"Theoretical answer: {answer}, labels give: {labeled_answer}")

Theoretical answer: Alexandria, labels give: Alexandria


In [ ]:
idx = 4
sample_idx = inputs["overflow_to_sample_mapping"][idx]
answer = answers[sample_idx]["text"][0]

decoded_example = tokenizer.decode(inputs["input_ids"][idx])
print(f"Theoretical answer: {answer}, decoded example: {decoded_example}")

Now let’s check index 4, where we set the labels to (0, 0), which means the answer is not in the context chunk of that feature
Indeed, we don’t see the answer inside the context.

In [22]:
max_length = 1024
stride = 128

def preprocess_training_dataset(examples):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["text"],
         max_length=max_length,
        truncation="only_second",
        stride=stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    offset_mapping = inputs.pop("offset_mapping")
    sample_map = inputs.pop("overflow_to_sample_mapping")
    answers = examples["short_candidate"]
    idxs = examples["str_idx"]
    start_positions = []
    end_positions = []

    for i, offset in enumerate(offset_mapping):
        sample_idx = sample_map[i]
        answer = answers[sample_idx]
        start_char = idxs[sample_idx]
        end_char = start_char + len(answer)
        sequence_ids = inputs.sequence_ids(i)

        # Find the start and end of the context
        idx = 0
        while sequence_ids[idx] != 1:
            idx += 1
        context_start = idx
        while sequence_ids[idx] == 1:
            idx += 1
        context_end = idx - 1

        # If the answer is not fully inside the context, label is (0, 0)
        if offset[context_start][0] > start_char or offset[context_end][1] < end_char:
            start_positions.append(0)
            end_positions.append(0)
        else:
            # Otherwise it's the start and end token positions
            idx = context_start
            while idx <= context_end and offset[idx][0] <= start_char:
                idx += 1
            start_positions.append(idx - 1)

            idx = context_end
            while idx >= context_start and offset[idx][1] >= end_char:
                idx -= 1
            end_positions.append(idx + 1)

    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    return inputs

In [23]:
train_dataset = train_ds.map(
    preprocess_training_dataset,
    batched=True,
    remove_columns=train_ds.column_names,
)
len(train_ds), len(train_dataset)

Map:   0%|          | 0/4000 [00:00<?, ? examples/s]

(4000, 4000)

In [24]:
def preprocess_validation_dataset(examples):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["text"],
        max_length=max_length,
        truncation="only_second",
        stride=stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    sample_map = inputs.pop("overflow_to_sample_mapping")
    example_ids = []

    for i in range(len(inputs["input_ids"])):
        sample_idx = sample_map[i]
        example_ids.append(examples["id"][sample_idx])

        sequence_ids = inputs.sequence_ids(i)
        offset = inputs["offset_mapping"][i]
        inputs["offset_mapping"][i] = [
            o if sequence_ids[k] == 1 else None for k, o in enumerate(offset)
        ]

    inputs["example_id"] = example_ids
    return inputs

In [25]:
test_dataset = test_ds.map(
    preprocess_validation_dataset,
    batched=True,
    remove_columns=test_ds.column_names,
)
len(test_ds), len(test_dataset)

Map:   0%|          | 0/989 [00:00<?, ? examples/s]

(989, 989)

Evaluation

In [26]:
#!pip install TensorFlow==2.14.0

In [27]:
from sklearn.metrics import f1_score

def compute_metrics(pred):
    # print(pred)
    start_labels = pred.label_ids[0]
    start_preds = pred.predictions[0].argmax(-1)
    end_labels = pred.label_ids[1]
    end_preds = pred.predictions[1].argmax(-1)


    f1_start = f1_score(start_labels, start_preds, average='macro')
    f1_end = f1_score(end_labels, end_preds, average='macro')


    return {
        'f1_start': f1_start,
        'f1_end': f1_end,
    }

Training

In [ ]:
!pip install transformers[torch]
!pip install accelerate -U


In [29]:
from transformers import Trainer, TrainingArguments
from transformers import AutoModelForQuestionAnswering

model = AutoModelForQuestionAnswering.from_pretrained("vinai/bartpho-word-base")

epochs = 10
batch_size = 4
lr = 2e-5

training_args = TrainingArguments(
    output_dir='results',          # output directory
    overwrite_output_dir=True,
    num_train_epochs=3,              # total number of training epochs
    per_device_train_batch_size=7,  # batch size per device during training
    per_device_eval_batch_size=7,   # batch size for evaluation
    warmup_steps=20,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir=None,            # directory for storing logs
    logging_steps=100,
    # label_names=['input_ids','attention_mask'] ,
    eval_steps = 200,
    # eval_accumulation_steps=1,
    # report_to='tensorboard',
    # save_total_limit=1,
    evaluation_strategy="steps",
    save_steps=200,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics
)
trainer.train()

Some weights of MBartForQuestionAnswering were not initialized from the model checkpoint at vinai/bartpho-word-base and are newly initialized: ['qa_outputs.weight', 'qa_outputs.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss,Validation Loss
200,2.369200,No log
400,1.793600,No log
600,1.410500,No log
800,1.066300,No log
1000,1.023100,No log
1200,0.690700,No log
1400,0.558300,No log
1600,0.559900,No log


TrainOutput(global_step=1716, training_loss=1.2700978070150166, metrics={'train_runtime': 4482.1306, 'train_samples_per_second': 2.677, 'train_steps_per_second': 0.383, 'total_flos': 7317177384960000.0, 'train_loss': 1.2700978070150166, 'epoch': 3.0})

In [30]:
trainer.evaluate(test_dataset)

{'eval_runtime': 114.9076,
 'eval_samples_per_second': 8.607,
 'eval_steps_per_second': 1.236,
 'epoch': 3.0}

Test Model

In [ ]:
from transformers import pipeline

# Replace this with your own checkpoint
model_checkpoint = "results/checkpoint-1000"

question_answerer = pipeline("question-answering", model=model_checkpoint, tokenizer = tokenizer)

context = "có 23 quốc gia không có lực lượng quân đội, bao gồm: Costa Rica, Iceland, Panama, Micronesia, Quần đảo Marshall, và Vatican..."

question = "Đất nước nào không có quân đội"
question_answerer(question=question, context=context)